In [104]:
import pickle
import matplotlib.pyplot as plt
from ctypes import *
import numpy as np
import base64
import matplotlib
from collections import Counter
import os
import sys

In [105]:
from lib.proto.protocol_pb2 import *
from google.protobuf.json_format import MessageToJson, ParseDict, MessageToDict
from lib.proto.protocol_pb2 import DatumTypeID, LinkID, TalkerID
import lib.proto.protocol_pb2 as protocol
from lib.comms_lib.crctabgen import crc16
# from link import cobs_decode

In [106]:
def loadlog(filename):
    segments = pickle.load(open(filename, 'rb'))
    segments, ack = segments[:-1], segments[-1]
    # print(ack)
    crc_xor = ack['log_crc16']
    xor = 0
    for s in segments:
        xor ^= s['segment_crc16']
    if (xor == crc_xor):
        # print("Data valid!")
        pass
    else:
        print(f"Data CRC mismatch: {filename}")
    return segments

In [107]:
logfiles = [
    'autolog_test.pickle',
	'board_2_shake_test.pickle',
	'desk_long_log.pickle',
	'log_50hz_walking.pickle',
	'log_acc_cal.pickle',
	'log_steadystate.pickle',
	'prelog_test.pickle',
	'tripoli_flight_1_h169_10.pickle',
	'tripoli_flight_1_h169_10_dl2.pickle',
	'tripoli_flight_2_h219_crash_dl2.pickle',
	# 'tripoli_flight_3_orange_h550.pickle',
	'tripoli_launch_4_i500.pickle',
	'vertical_shake_test_tripoli.pickle',
	'walk_on_street.pickle'
]

In [108]:
LOGGER_COBS_ESC = 0xAA
LOGGER_COBS_ESC_AA = 0x01
LOGGER_COBS_ESC_FF = 0x02


def cobs_decode(data: bytes) -> bytes:
    dout = bytes()
    diter = iter(data)
    while (d := next(diter, None)) is not None:
        if (d == LOGGER_COBS_ESC):
            dout += bytes([LOGGER_COBS_ESC if next(diter, None)
                          == LOGGER_COBS_ESC_AA else 0xFF])
        else:
            dout += bytes([d])
    return dout

In [109]:
def parselog(segments):
    databuffer = bytes()
    for s in segments:
        databuffer += base64.b64decode(s['data'])
    frames = databuffer.split(b'\xFF')
    frames_dec = [e for e in [cobs_decode(f) for f in frames] if len(e)]
    return frames_dec

logs = [(log,parselog(loadlog(log))) for log in logfiles]

Data CRC mismatch: tripoli_flight_1_h169_10.pickle
Data CRC mismatch: tripoli_flight_1_h169_10_dl2.pickle
Data CRC mismatch: tripoli_flight_2_h219_crash_dl2.pickle
Data CRC mismatch: tripoli_launch_4_i500.pickle


In [110]:
for name, log in logs:
    print(name, Counter([len(f) for f in log]))

autolog_test.pickle Counter({72: 9173})
board_2_shake_test.pickle Counter({72: 2213})
desk_long_log.pickle Counter({72: 11647})
log_50hz_walking.pickle Counter({72: 4180})
log_acc_cal.pickle Counter({72: 7753})
log_steadystate.pickle Counter({72: 4461})
prelog_test.pickle Counter({72: 2159})
tripoli_flight_1_h169_10.pickle Counter({72: 19202, 76: 1})
tripoli_flight_1_h169_10_dl2.pickle Counter({72: 19257, 92: 1})
tripoli_flight_2_h219_crash_dl2.pickle Counter({72: 8212, 28: 1})
tripoli_launch_4_i500.pickle Counter({72: 119309, 74: 2, 97: 1, 82: 1, 114: 1})
vertical_shake_test_tripoli.pickle Counter({72: 498})
walk_on_street.pickle Counter({72: 13759})
